# 🚀 CI/CD Pipeline for .NET RAG App using Azure DevOps

This notebook provides a detailed walkthrough of setting up an end-to-end **CI/CD pipeline using Azure DevOps** for a containerized **.NET + Blazor RAG system**.

We cover:
- YAML pipeline definition
- Build & test stages
- Docker image build & publish to Azure Container Registry (ACR)
- Kubernetes or App Service deployment (optional)

## 📁 Project Structure Overview
```bash
rag-app/
├── RAGWeb/                # .NET Blazor project
├── Dockerfile             # Dockerfile for app
├── docker-compose.yml
└── azure-pipelines.yml    # CI/CD pipeline (see below)
```

## 🧾 azure-pipelines.yml
Place this in your project root as `azure-pipelines.yml`.
Ensure your Azure DevOps project is connected to your Git repo and ACR.

In [ ]:
trigger:
  branches:
    include:
      - main

variables:
  dockerRegistryServiceConnection: 'YOUR_ACR_SERVICE_CONNECTION'
  imageRepository: 'ragweb'
  containerRegistry: 'youracr.azurecr.io'
  dockerfilePath: 'Dockerfile'
  tag: '$(Build.BuildId)'

stages:
- stage: Build
  jobs:
  - job: Build
    pool:
      vmImage: 'ubuntu-latest'
    steps:
    - task: UseDotNet@2
      inputs:
        packageType: 'sdk'
        version: '7.0.x'
    
    - task: DotNetCoreCLI@2
      displayName: 'Restore NuGet'
      inputs:
        command: 'restore'
        projects: '**/*.csproj'

    - task: DotNetCoreCLI@2
      displayName: 'Build'
      inputs:
        command: 'build'
        arguments: '--configuration Release'

    - task: DotNetCoreCLI@2
      displayName: 'Test'
      inputs:
        command: 'test'
        projects: '**/*Tests.csproj'


In [ ]:
- stage: Deploy
  jobs:
  - job: DockerBuildAndPush
    displayName: 'Build and Push Docker Image'
    pool:
      vmImage: 'ubuntu-latest'
    steps:
    - task: Docker@2
      displayName: 'Build and Push'
      inputs:
        command: 'buildAndPush'
        repository: '$(imageRepository)'
        dockerfile: '$(dockerfilePath)'
        containerRegistry: '$(dockerRegistryServiceConnection)'
        tags: 'latest,$(tag)'


## 🌐 Optional: Azure App Service or AKS Deployment

You can add a third stage to deploy to:
- Azure Kubernetes Service (AKS)
- Azure App Service for Containers

**For AKS**: Use `kubectl apply -f k8s.yaml` in a Bash task
**For App Service**: Use `AzureWebAppContainer@1` task

## 🔐 Secure Secrets
- Use Azure DevOps **Library** to store secrets like `OPENAI_API_KEY`
- Reference them in pipeline with `env:` or `--build-arg`
- For App Service: Use Configuration Settings (App Settings)
- For AKS: Use Kubernetes secrets or Azure Key Vault integration